In [0]:
from __future__ import print_function
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.datasets import imdb
from matplotlib import pyplot as plt

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [0]:

max_features = 20000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 100
batch_size = 32
index_from=3


## Загрузим данные
Нам нужны только 20к самых популярных слов

Остальное забьем токеном UNK, он же OOV

In [0]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features, index_from=3)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Вот так выглядят закодированные отзывы

In [0]:
print(x_train[5]) 

## Паддинг
В целях эффективности(выделение памяти под тензоры, работа с ними) рекуррентные сетки принято кормить батчами последовательностей одной длины.

Можно, конечно, попробовать побить их на батчи примерно одной длины, чтобы паддить меньше, но мы будем решать задачу в лоб:

In [0]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

## Построение индекса слов

Эта штука нам пригодится в продакшне, а так же для отладки.

Keras резерзвирует 0 как специальный символ для паддинга в своих Embedding-слоях.


*   В датасете IMDB 1 занята специальным токеном "начало предложения"
*   2 занята токеном UNK/OOV
*   3 не используется в виду ошибки в коде(https://jamesmccaffrey.wordpress.com/2018/04/27/inspecting-the-imdb-dataset-reverse-mapping-the-index-values/)


учтем это при построении индекса: сместим все индексы на 3



In [0]:
word_to_id = imdb.get_word_index()
word_to_id = {k:(v+index_from) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
word_to_id["<UNUSED>"] = 3

id_to_word = {value:key for key,value in word_to_id.items()}

## Конструируем модельку



1.   Эмбеддинги по количеству токенов(20к + 4 специальных)
2.   Двунаправленный LSTM
3.   Дропаут(она все равно переобучится в хлам, но мы "пока об этом не знаем")
4.   Логистическая регрессия, классифицирующая стейт LSTMки




In [0]:
model = Sequential()
model.add(Embedding(max_features + index_from + 1, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


In [0]:
# try using different optimizers and different optimizer configs
from keras.callbacks import History
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])
print(model.summary())

## Оверфитнем все
No comments

In [0]:
print('Train...')
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=4,
          validation_data=[x_test, y_test])

In [0]:
plt.figure(figsize=(15,10))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [0]:
print(' '.join(id_to_word[id] for id in x_train[4])) ## Проверка, что wordindex построен правильно

## Выгружаем модель и индекс

In [0]:
model.save('clf.h5')

In [0]:
import json
with open('word_index.json', 'w') as f:
  f.write(json.dumps(word_to_id))

In [0]:
import keras
keras.backend.floatx() ## Лучше бы знать, сколькибитный float использовал keras на обучении. Потом можно наловить с этим проблем

# Идеи для улучшешия


*   Использовать взрослый препроцессинг(лемматизация/стемминг)
*   Использовать предобученные word embeddings(fasttext, glove, word2vec)  (упороться и предобучить самостоятельно?)
*   Доучить предобученные эмбеддинги
*   Доучить предобученные эмбеддинги, не испортив их
*   Более разумно формировать словарь(подсказка: посмотрите на первое слово в индексе после специальных токенов. Они отсортированы по популярности)
*   Побороть переобучение(обратите внимание на train/test split и ширину LSTM)

